In [89]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
SEED=2
import os

train=pd.read_csv("/kaggle/input/aptos2019-blindness-detection/train.csv")
submition=pd.read_csv("/kaggle/input/aptos2019-blindness-detection/test.csv")

x = train['id_code']
y = train['diagnosis']

x,y=shuffle(x,y)
# Any results you write to the current directory are saved as output.

In [90]:
df_X,X_test,df_y,y_test=train_test_split(x, y, test_size=0.15)

X_train,X_valid,y_train,y_valid=train_test_split(df_X, df_y, test_size=0.15)


In [91]:
def load_ben_color(path, sigmaX=20):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [92]:
def crop_image1(img,tol=7):
    # img is image data
    # tol  is tolerance
        
    mask = img>tol
    return img[np.ix_(mask.any(1),mask.any(0))]

def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
    #         print(img1.shape,img2.shape,img3.shape)
            img = np.stack([img1,img2,img3],axis=-1)
    #         print(img.shape)
        return img

In [93]:
import matplotlib.pyplot as plt 
import cv2
train_images=[]
IMG_SIZE=224
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_train):
    
    path=f"../input/aptos2019-blindness-detection/train_images/{row}.png"
    image =load_ben_color(path)
    train_images.append(image)

        

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.58 µs


<Figure size 1800x1152 with 0 Axes>

In [94]:
import matplotlib.pyplot as plt 
import cv2
test_images=[]
IMG_SIZE=224
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_test):
    
    path=f"../input/aptos2019-blindness-detection/train_images/{row}.png"
    image =load_ben_color(path)
    test_images.append(image)


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.82 µs


<Figure size 1800x1152 with 0 Axes>

In [95]:
import matplotlib.pyplot as plt 
import cv2
valid_images=[]
IMG_SIZE=224
%time
fig = plt.figure(figsize=(25, 16))

for idx, row in enumerate(X_valid):
    
    path=f"../input/aptos2019-blindness-detection/train_images/{row}.png"
    image =load_ben_color(path)
    valid_images.append(image)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 9.06 µs


<Figure size 1800x1152 with 0 Axes>

In [96]:

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import (Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D,
                          BatchNormalization, Input, Conv2D, GlobalAveragePooling2D)
from keras.applications.resnet50 import ResNet50
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model

In [155]:
from keras.utils import to_categorical
y_train_dummies=to_categorical(y_train)
y_test_dummies=to_categorical(y_test)
y_valid_dummies=to_categorical(y_valid)
y_train_dummies

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]], dtype=float32)

In [156]:
import numpy as np
import time
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
import os




In [157]:
num_Classes=5

train=np.array(train_images)
test=np.array(test_images)
valid=np.array(valid_images)

num_of_samples=train.shape[0]
train.shape

(2343, 224, 224, 3)

In [158]:
BATCH_SIZE = 4

# Add Image augmentation to our generator
datagen=ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)


In [159]:
from keras.preprocessing import image                  
image_input=Input(shape=train.shape[1:])
model=VGG16(input_tensor=image_input, weights='imagenet')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [160]:


last_layer=model.get_layer('fc1').output
last_layer=Dropout(0.2)(last_layer)
out=Dense(num_Classes,activation='softmax', name='output')(last_layer)

custom_vgg_model=Model(image_input,out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [161]:
for layer in custom_vgg_model.layers[:-2]:
    layer.trainable=False

In [162]:
custom_vgg_model.compile(loss="categorical_crossentropy", optimizer='Adam',  metrics=['accuracy'])

In [163]:

datagen.fit(train)

t=time.time()

custom_vgg_model.fit_generator(datagen.flow(train,y_train_dummies, batch_size=32),steps_per_epoch=len(X_train), epochs=3, verbose=1 ,validation_data=(valid,y_valid_dummies))

print("training time: %s" %(t-time.time()))

Epoch 1/3
2343/2343 [==============================] - 717s 306ms/step - loss: 0.8785 - acc: 0.7026 - val_loss: 4.0367 - val_acc: 0.6553
Epoch 2/3
2343/2343 [==============================] - 713s 304ms/step - loss: 0.7644 - acc: 0.7384 - val_loss: 5.4120 - val_acc: 0.6109
Epoch 3/3
2343/2343 [==============================] - 715s 305ms/step - loss: 0.7457 - acc: 0.7463 - val_loss: 6.4427 - val_acc: 0.5648
training time: -2145.83038187027


In [164]:


t=time.time()

custom_vgg_model.fit(train,y_train_dummies, batch_size=32, epochs=30, verbose=1 ,validation_data=(valid,y_valid_dummies))

print("training time: %s" %(t-time.time()))

Train on 2343 samples, validate on 586 samples
Epoch 1/30
2343/2343 [==============================] - 7s 3ms/step - loss: 4.2548 - acc: 0.7017 - val_loss: 4.2430 - val_acc: 0.7167
Epoch 2/30
2343/2343 [==============================] - 7s 3ms/step - loss: 4.0679 - acc: 0.7217 - val_loss: 3.8921 - val_acc: 0.7321
Epoch 3/30
2343/2343 [==============================] - 7s 3ms/step - loss: 3.7460 - acc: 0.7452 - val_loss: 3.6728 - val_acc: 0.7526
Epoch 4/30
2343/2343 [==============================] - 7s 3ms/step - loss: 3.5540 - acc: 0.7571 - val_loss: 3.5269 - val_acc: 0.7628
Epoch 5/30
2343/2343 [==============================] - 7s 3ms/step - loss: 3.4531 - acc: 0.7601 - val_loss: 3.5515 - val_acc: 0.7577
Epoch 6/30
2343/2343 [==============================] - 7s 3ms/step - loss: 3.4114 - acc: 0.7644 - val_loss: 3.3427 - val_acc: 0.7713
Epoch 7/30
2343/2343 [==============================] - 7s 3ms/step - loss: 3.3246 - acc: 0.7644 - val_loss: 3.3430 - val_acc: 0.7611
Epoch 8/30
2343

In [165]:
(loss, accuracy)=custom_vgg_model.evaluate(test,y_test_dummies,verbose=1 ,batch_size=32)
print(loss)
print(accuracy*100)

733/733 [==============================] - 2s 2ms/step
3.2996104137634026
73.2605730202481


In [166]:
predicted=[]
submit = pd.read_csv('../input/aptos2019-blindness-detection/sample_submission.csv')
for i, name in tqdm(enumerate(submit['id_code'])):
    path = os.path.join('../input/aptos2019-blindness-detection/test_images/', name+'.png')
    image = cv2.imread(path)
    image = cv2.resize(image, (224, 224))
    score_predict = model.predict((image[np.newaxis])/255)
    label_predict = np.argmax(score_predict)
    # label_predict = score_predict.astype(int).sum() - 1
    predicted.append(str(label_predict))

1928it [00:56, 34.28it/s]


In [167]:
submit['diagnosis'] = predicted
submit.to_csv('submission.csv', index=False)
submit.head()

,id_code,diagnosis
0,0005cfc8afb6,669
1,003f0afdcd15,669
2,006efc72b638,669
3,00836aaacf06,669
4,009245722fa4,669
